# Análisis del lenguaje humano: Equidad & Word Embedding

## 1. Introducción

- __Los humanos tienden a querer diferenciarse entre distintos grupos (por ciertos motivos de percepción del mundo)__ ...

...por nacionalidad o origen etnico

...por religión

...por color de piel

...por ideología política

...por genero u orientación sexual

...por nivel socioeconómico

...por el aspecto físico

...por ciertas preferencias o decisiones ("antivacunas" vs. "provacunas", "carnivoros" vs. "vegetarianos")

...etc.

- __Los humanos tienden a asociar estereotipos positivos o negativos a ciertos grupos (por ciertos motivos)__


Por ejemplo: 

"los franceses son ladrones"

"los cristianos son violentos"

"las mujeres son fragiles"

"los ricos son corruptos"
     

- __Los estereotipos pueden derivar hacia discurso de odio, situaciones injustas y violencia__

Podemos pensar por ejemplo a los estereotipos sobre los judios al siglo XX que llevaron a la violenca que conocemos.

- __Los "discursos" humanos reflejan nuestros estereotipos / nuestra cultura.__

¿El discurso de los medios de prensa es un espejo de nuestros estereotipos/nuestra cultura?

- __Todos las técnicas de NLP "modernas" se basan sobre modelos de Word Embedding que capturan el sentido de cada palabra a través de un vector__

Hemos visto que estos vectores capturan tambien nuestros estereotipos. Por ejemplo, vimos que el vector de la palabra "ingenieria" es más similar al vector de "hombre" que al vector de "mujer".



- __Preguntas__:

¿Es razonable que las técnicas de IA/NLP se basen sobre modelos que reflejan nuestros estereotipos culturales?

Las técnicas de IA/NLP no tienen capacidades de pensamiento crítico sobre su propio conocimiento, ¿pero pueden ayudarnos como humanos a mejorar nuestro pensamiento crítico?

¿Cuáles son los estereotipos culturales capturados por las técnicas de Word Embedding? ¿Se puede observar evoluciones?


## 2. El framework WEFE: Word Embedding Fairness Evaluation

El framework WEFE es una librería abierta desarrollada en el DCC de la U. de Chile. Encapsula distintos métodos y métricas para medir qué tan sesgado es un modelo de Word Embedding (o diciendo de otra manera: medir si el modelo contiene estereotipos sobre grupos de humanos)

Documentación: https://wefe.readthedocs.io/en/latest/

Referencia: P. Badilla, F. Bravo-Marquez, and J. Pérez WEFE: The Word Embeddings Fairness Evaluation Framework In Proceedings of the 29th International Joint Conference on Artificial Intelligence and the 17th Pacific Rim International Conference on Artificial Intelligence (IJCAI-PRICAI 2020), Yokohama, Japan.

Enlace: https://felipebravom.com/publications/ijcai2020.pdf


In [4]:
!pip install wefe

Una de las principales métricas propuestas para medir la (in)equidad en el lenguaje humano se llama Relative Norm Distance (RND). Se basa en los conceptos siguientes:

- Los __targets__ : un target es un grupo humano y las distintas palabras existentes para referise a ese grupo humano.

Por ejemplo, para hablar de los "chilenos" como grupo humano, puedo utilizar las palabras "chilenos" y "chilenas".
Por ejemplo, para hablar de los "brasileños" como grupo humano, puedo utilizar las palabras "brasileños" y "brasileñas".


- Los __atributos__ : un atributo corresponde a un concepto semántico las distintas palabras existentes para referirse a ella.

Por ejemplo, la "violencia" es un concepto semántico y se referise a ella con distintas palabras como "agresión", "crueldad", "violación", etc.

A partir de distintos _targets_, la métrica RND calcula una distancia promedia de los _targets_ con un atributo. Permite identificar si el discurso humano tiende a asociar el concepto semántico hacia cierto grupo humano.


Veamos un ejemplo...

In [13]:
from wefe.query import Query
from wefe.word_embedding_model import WordEmbeddingModel
from wefe.metrics.RND import RND
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

Cargamos un modelo de Word Embedding entrenado a partir de 1 año de noticias de prensa chilena (2020).

Este modelo fue entrenado por Nicolas Treimun (estudiante de la carrera ICI).

In [70]:
name_model = "Chile/CHILE_2020_dev210721_113361_lg.model"

model = KeyedVectors.load(name_model)                                

Definemos los grupos humanos (__targets__) y el concepto semántico (__atributo__) que queremos evaluar.

In [22]:
target_sets = [['chilenos', 'chilenas'],['brasileños','brasileñas']]
target_sets_names = ['chilenxs', 'brasileñxs']

In [26]:
attribute_sets = [['violencia', 'terrorismo', 'crueldad', 'violación']]
attribute_sets_names = ['violencia']

Se agrupa estas variables para definir una consulta y evaluar el modelo de Word Embedding con la métrica de RND.

In [45]:
query = Query(target_sets, attribute_sets, target_sets_names, attribute_sets_names)

In [46]:
word_vectors = model.wv
mdl = WordEmbeddingModel(word_vectors)
rnd = RND()
result_rnd = rnd.run_query(query, mdl)
print(result_rnd)

{'query_name': 'hombres and mujeres wrt dulzura', 'result': -1.1469440460205078, 'rnd': -1.1469440460205078, 'distances_by_word': {'dulzura': -1.146944}}


- un resultado de 0 es ideal. Significa que el concepto semantico se encuentra a equidistancia entre los grupos humanos

- un resultado positivo significa que el concepto semántico se encuentra más vinculado al primer grupo definido.

- un resultado negativo significa que el concepto semántico se encuentra más vinculado al segundo grupo definido

Veamos un segundo ejemplo...

In [71]:
target_sets = [['hombres','hombre'],['mujeres','mujer']]
target_sets_names = ['hombres', 'mujeres']

In [72]:
attribute_sets = [['dulzura','bondad','ternura','generosidad','inteligencia','empatía','fuerza']]
attribute_sets_names = ['características humanas']

In [73]:
query = Query(target_sets, attribute_sets, target_sets_names, attribute_sets_names)
word_vectors = model.wv
mdl = WordEmbeddingModel(word_vectors)
rnd = RND()
result_rnd = rnd.run_query(query, mdl)
print(result_rnd)

{'query_name': 'hombres and mujeres wrt características humanas', 'result': 0.11398083823067802, 'rnd': 0.11398083823067802, 'distances_by_word': {'dulzura': -1.146944, 'empatía': -0.20008087, 'ternura': -0.04829979, 'generosidad': 0.16884995, 'bondad': 0.32925415, 'inteligencia': 0.5942173, 'fuerza': 1.1008692}}


Comparemos el año 2020 con el año 2018...

In [74]:
name_model = "Chile/CHILE_2018_dev210721_110981_lg.model"

model = KeyedVectors.load(name_model)  

In [77]:
target_sets = [['hombres','hombre'],['mujeres','mujer']]
target_sets_names = ['hombres', 'mujeres']

attribute_sets = [['dulzura','bondad','ternura','generosidad','inteligencia','empatía','fuerza']]
attribute_sets_names = ['características humanas']

query = Query(target_sets, attribute_sets, target_sets_names, attribute_sets_names)
word_vectors = model.wv
mdl = WordEmbeddingModel(word_vectors)
rnd = RND()
result_rnd = rnd.run_query(query, mdl)
print(result_rnd)

{'query_name': 'hombres and mujeres wrt características humanas', 'result': -0.7215265546526227, 'rnd': -0.7215265546526227, 'distances_by_word': {'dulzura': -1.7727661, 'bondad': -1.2733221, 'empatía': -0.8307848, 'ternura': -0.59896374, 'generosidad': -0.57681847, 'inteligencia': -0.454628, 'fuerza': 0.45659733}}


Comparemos Chile con España...

In [78]:
name_model = "España/ESPANA_2020_dev300721_NON_lg.model"

model = KeyedVectors.load(name_model) 

In [80]:
name_model = "España/ESPANA_2017_dev300721_NON_lg.model"

model = KeyedVectors.load(name_model) 

In [81]:
target_sets = [['hombres','hombre'],['mujeres','mujer']]
target_sets_names = ['hombres', 'mujeres']

attribute_sets = [['dulzura','bondad','ternura','generosidad','inteligencia','empatía','fuerza']]
attribute_sets_names = ['características humanas']

query = Query(target_sets, attribute_sets, target_sets_names, attribute_sets_names)
word_vectors = model.wv
mdl = WordEmbeddingModel(word_vectors)
rnd = RND()
result_rnd = rnd.run_query(query, mdl)
print(result_rnd)

{'query_name': 'hombres and mujeres wrt características humanas', 'result': -0.156604494367327, 'rnd': -0.156604494367327, 'distances_by_word': {'dulzura': -0.7331772, 'bondad': -0.41804314, 'generosidad': -0.24637985, 'ternura': -0.2076807, 'empatía': -0.16707897, 'fuerza': 0.14738083, 'inteligencia': 0.52874756}}


## 3. Experimento en grupo

¿Cómo evaluar la equidad en el lenguaje humano?

1) Elegir un problema de (in)equidad, identificando grupos humanos y concepto semántico

2) Identificar palabras para referise a los grupos humanos y concepto semántico

3) Evaluar el nivel de (in)equidad) en distintos paises y años
